In [3]:
import numpy as np
from collections import Counter
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [4]:
class DecisionTree:
    def __init__(self, d=None):
        self.max_depth = d
        self.tree = None


    def fit(self, X, y):
        self.tree = self._build_tree(X, y)


    def predict(self, X):
        return np.array([self.pred(inputs) for inputs in X])


    def _gini(self, y):
        #calculating the gini impurity
        m = len(y)
        return 1.0 - sum((np.sum(y == c) / m) ** 2 for c in np.unique(y))


    def _split(self, X, y, index, value):
        #splitting the node
        left_mask = X[:, index] <= value
        right_mask = X[:, index] > value
        return X[left_mask], X[right_mask], y[left_mask], y[right_mask]


    def _best_split(self, X, y):
        #initializing the gini value to infinity (worst case)
        best_gini = float('inf')
        best_index, best_value = None, None

        for index in range(X.shape[1]):
            #looping through all the unique values in the column
            for value in np.unique(X[:, index]):
                #splitting
                X_left, X_right, y_left, y_right = self._split(X, y, index, value)
                #checking if either splitted node is empty i.e. the splitting didn't occur
                if len(y_left) == 0 or len(y_right) == 0:
                    continue
                #calculating the gini impurity
                gini = (len(y_left) * self._gini(y_left) + len(y_right) * self._gini(y_right)) / len(y)
                #updating the best gini and the value that it belongs to
                if gini < best_gini:
                    best_gini, best_index, best_value = gini, index, value
        return best_index, best_value


    def _build_tree(self, X, y, depth=0):
        #condition that terminates the recursive building of the tree
        if len(np.unique(y)) == 1 or (self.max_depth and depth >= self.max_depth):
            return np.argmax(np.bincount(y))

        #finding the best attribute to make the node that we would split
        index, value = self._best_split(X, y)
        if index is None:
            return np.argmax(np.bincount(y))

        #splitting the node into left and right sub nodes
        X_left, X_right, y_left, y_right = self._split(X, y, index, value)

        #recursively creating the left subtree
        left_subtree = self._build_tree(X_left, y_left, depth + 1)
        #recursively creating the right subtree
        right_subtree = self._build_tree(X_right, y_right, depth + 1)

        return (index, value, left_subtree, right_subtree)


    def pred(self, inputs):
        node = self.tree
        while isinstance(node, tuple):
            if inputs[node[0]] <= node[1]:
                node = node[2]
            else:
                node = node[3]
        return node


In [5]:
class RandomForest:
    def __init__(self, trees=100, d=None, bt=True, f='sqrt'):
        self.no_of_trees = trees
        self.max_depth = d
        self.bootstrap = bt
        self.max_features = f
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        no_of_samples, no_of_features = X.shape

        for _ in range(self.no_of_trees):
            #creating different samples of data using bootstrap sampling technique
            #creating random samples of data to create different kinds of trees
            if self.bootstrap:
                indices = np.random.choice(no_of_samples, no_of_samples, replace=True)
            else:
                indices = np.arange(no_of_samples)

            #selecting random features for each tree to be splitted upon
            #values close to the log and square root of the total no of features work the best
            if self.max_features == 'sqrt':
                max_features = int(np.sqrt(no_of_features))
            elif self.max_features == 'log2':
                max_features = int(np.log2(no_of_features))
            else:
                max_features = no_of_features

            #dividng the sample into x and y subsets
            feature_indices = np.random.choice(no_of_features, max_features, replace=True)
            X_subset = X[indices][:, feature_indices]
            y_subset = y[indices]

            #training the tree
            tree = DecisionTree(d=self.max_depth)
            tree.fit(X_subset, y_subset)
            self.trees.append((tree, feature_indices))

    def predict(self, X):
        #collecting the predictions from each of the tree
        tree_preds = np.array([tree.predict(X[:, features]) for tree, features in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        #finding the mode i.e. the most common prediction among all the predictions collected
        #in other words, finding the aggregate result
        y_pred = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]
        #creating the result into an array
        return np.array(y_pred)

In [9]:
# Example usage:
if __name__ == "__main__":
    #loading the Iris dataset
    iris = load_iris()
    X, y = iris.data, iris.target

    #splitting the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    #creating & training the Random Forest
    rf = RandomForest(trees=10, d=3)
    rf.fit(X_train, y_train)

    predictions = rf.predict(X_test)

    print("Predictions:  ", predictions)
    print("Actual labels:", y_test)


Predictions:   [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
Actual labels: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0]
